In [17]:
from __future__ import division
from nltk.stem.snowball import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
from datetime import datetime
import nltk
import numpy as np
import string
import csv
import sys
import os
from importlib import reload

In [18]:
"""
TODO: STORE SENTIMENT INFO IN A WAY THAT ALLOWS US TO CREATE RUNNING SENTIMENT GRAPHS

- Cut text into 20 or so roughly equal sized chunks of sentences
- Store mean positive and negative sentiment values for each segment as a feature 
- Create graphs of sentiment movement for data viz (upper bound == max value observed, lower bound == 0)

"""
#need this or else it throws encoding/decoding errors
reload(sys)
# sys.setdefaultencoding('utf8')
punct = set(['!', '#', '"', '%', '$', '&', '(', '+', '*', '-', ',', '/', '.', ';', ':', '=', '<', '?', '>', '@', '[', ']', '_', '^', '`', '{', '~'])
table = str.maketrans("","")
target = open("output_POS.csv", 'w')

#check avg sent size
target.write("book_name|total_words|avg_sentence_size|!|#|''|%|$|&|(|+|*|-|,|/|.|;|:|=|<|?|>|@|[|]|_|^|`|{|~|neg|neu|pos|compound|ID|Title|Author|CC|CD|DT|EX|FW|IN|JJ|JJR|JJS|LS|MD|NN|NNP|NNPS|NNS|PDT|PRP|PRP$|RB|RBR|RBS|RP|VB|VBD|VBG|VBP|VBN|WDT|VBZ|WRB|WP$|WP|")
target.write('\n')

1

In [19]:
def punctAndWordsInSentence(listOfCharacters):
    punctuation_dict = {}
    sentenceCounter = 0;
    wordCounter = 0
    periodCounter = 0.1;
    avgSentenceSize = 0;
    totalWords = 0;
    punctCounter = 0;
    """
    Iterate through all characters. Count periods, punct frequencies. WordCounter = words in sentence (resets
    to zero after a period). totalWords is the book's total word count.
    """
    #sentence count
    for i in range(len(listOfCharacters)):
        if i != 0:
            #if lettter followed by space or punct, then word count +=1
            if (listOfCharacters[i] == " " or str(listOfCharacters[i]) in punct) and str(listOfCharacters[i-1]) in (string.ascii_lowercase + string.ascii_uppercase):
                totalWords = totalWords + 1
            #count periods as well
            if listOfCharacters[i] == ".":
                periodCounter = periodCounter + 1
            if listOfCharacters[i] in punct:
                punctCounter = punctCounter + 1
                if listOfCharacters[i] in punctuation_dict:
                    punctuation_dict[listOfCharacters[i]] = punctuation_dict[listOfCharacters[i]] + 1
                else:
                    punctuation_dict[listOfCharacters[i]] = 1



    avgSentenceSize = (totalWords/periodCounter)
    #put together output, bar delimited
    target.write(str(totalWords) + "|")
    target.write(str(avgSentenceSize) + "|")
    
    for i in punct:
        s = ""
        if i in punctuation_dict:
            s = s + str(punctuation_dict[i] / punctCounter) + "|"    #pct of punct that is [x]
        else:
            s = s + str(0) + "|"                                     #0 if unused
        target.write(s)

In [20]:
def pos_tagging(content):
    parts = ["CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNP", "NNPS", "NNS", "PDT", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "VB", "VBD", "VBG",  "VBP", "VBN", "WDT", "VBZ", "WRB", "WP$", "WP" ]
    content = str(content)   #see if this fixes the error
    text = nltk.word_tokenize(content)  #need to tokenize first
    results = nltk.pos_tag(text)
    #dict of {POS: count}
    results_dict = {}
    counter = 0
    for tag in results:
        token = tag[0]
        pos = tag[1]
        counter += 1
        if pos in results_dict:
            results_dict[pos] += 1
        else:
            results_dict[pos] = 1
    #write to file
    for i in parts:
        s = ""
        if i in results_dict:
            s = s + str(results_dict[i]/float(counter)) + "|"    #pct of POS that are [x]
        else:
            s = s + str(0) + "|"                                 #0 if unused
        target.write(s)

In [21]:
def decode_file(text):
    return text.decode('utf-8', errors='replace')

def get_title_author(text):
    author = "NULL"
    title = "NULL"
    text = text.splitlines()
    #for line in text, check if title or author stored there
    for i in range(80):
        #error handling since some texts are <80 lines
        try:
            if "Title: " in text[i]:
                title = text[i][7:]
            if "Author: " in text[i]:
                author = text[i][8:]
            #if they have both been found, do not waste extra time iterating 
            if title != "NULL" and author != "NULL":
                title_author_tuple = (title, author)
                return title_author_tuple
        except:
            pass
    title_author_tuple = (title, author)
    return title_author_tuple

In [22]:
def get_sentiment(temp):
    temp = b"temp.replace('\n', '')"
    temp = b"temp.replace('\r', '')"
    content = decode_file(temp)
    content = tokenize.sent_tokenize(content)
    #get author and title now that content is split by sentence 
    sid = SentimentIntensityAnalyzer()
    booksent = []
    for sentence in content:
        ss = sid.polarity_scores(sentence)
        ssarray = [ss['neg'],ss['neu'],ss['pos'], ss['compound']]
        booksent.append(ssarray)
    valuearray = np.array(booksent)
    # mean negative, neutral, positive, compound score for all lines in book
    values = np.mean(valuearray, axis=0)
    return values

In [32]:
def preprocessing():
    '''
    read file as a list of words
    set lowercase, stem, remove stopwords
    get punctuation string for later feature extraction
    save local wordcount dict
    save global word dict after finished looping through docs
    '''
    counter = 0
    cnt=0
    for book in os.listdir("C:/Users/krgau/OneDrive/Desktop/data/new/new1/"):
        if cnt<2500:
            if not book.startswith('.'):    #pass hidden files such as .DS_STORE
                book = str(book)
                with open("C:/Users/krgau/OneDrive/Desktop/data/new/new1/" + book, 'rb') as f:
                    content =b"f.read().rstrip('\n')"
                target.write(book + "|")
                punctAndWordsInSentence(content)
                sentiment_values = get_sentiment(content)
                neg = sentiment_values[0]
                neu = sentiment_values[1]
                pos = sentiment_values[2]
                compound = sentiment_values[3]
                target.write(str(neg) + "|" + str(neu) + "|" + str(pos) + "|" + str(compound) + "|" + str(counter) + "|")
                title_author_tuple = get_title_author(content)
                target.write(str(title_author_tuple[0]) + "|" + str(title_author_tuple[1]) + "|")
                pos_tagging(content)
                target.write('\n')
                f.close()
                counter += 1
                cnt=cnt+1
                if counter%20 == 0:
                    print ("book " + str(counter) + " done: " + book)
        else:
            break



In [33]:
startTime = datetime.now()
preprocessing()
print (datetime.now() - startTime)

book 20 done: 10014-8.txt
book 40 done: 10026-8.txt
book 60 done: 10037-8.txt
book 80 done: 10051.txt
book 100 done: 10070.txt
book 120 done: 10082.txt
book 140 done: 10104-8.txt
book 160 done: 10135-8.txt
book 180 done: 10151.txt
book 200 done: 10318-8.txt
book 220 done: 10356.txt
book 240 done: 10377.txt
book 260 done: 10404.txt
book 280 done: 10436.txt
book 300 done: 10448.txt
book 320 done: 10471-8.txt
book 340 done: 10494.txt
book 360 done: 10515.txt
book 380 done: 10535.txt
book 400 done: 10573.txt
book 420 done: 10594.txt
book 440 done: 10624-8.txt
book 460 done: 10647-8.txt
book 480 done: 10676-8.txt
book 500 done: 10711.txt
book 520 done: 10731.txt
book 540 done: 10754.txt
book 560 done: 10806-8.txt
book 580 done: 10845-8.txt
book 600 done: 10886.txt
book 620 done: 10911-8.txt
book 640 done: 10925-8.txt
book 660 done: 10938.txt
book 680 done: 10949.txt
book 700 done: 10961.txt
book 720 done: 10974-8.txt
book 740 done: 10988.txt
book 760 done: 11008.txt
book 780 done: 11023.txt